BERT에 비해 RoBERT가 더 많은 데이터로 사전학습하고 시간이 더 오래 걸림

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
pip install transformers

In [3]:
import pandas as pd
import numpy as np
from transformers import RobertaTokenizer, RobertaModel
import torch

# RoBERTa 모델 및 토크나이저 불러오기
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
model = RobertaModel.from_pretrained("roberta-base")

# CSV 파일 불러오기 (파일 경로는 상황에 맞게 조정하세요)
df = pd.read_csv('/content/drive/MyDrive/SKT_FLY_AI/Project/AI_Model/Similarity/law_add.csv', encoding='cp949')

# 사용자 입력 텍스트 예시
user_input_text = "부동산을 통해서 어제 전세 계약을 했는데 사기를 당함. 보증금을 돌려받고자 했으나 인정이 되지 않는 상황"

# 각 문장을 RoBERTa 임베딩으로 변환
encoded_inputs = tokenizer(df['fact_one'].tolist(), padding=True, truncation=True, return_tensors="pt")
user_input_encoded = tokenizer(user_input_text, padding=True, truncation=True, return_tensors="pt")

with torch.no_grad():
    outputs = model(**encoded_inputs)
    user_output = model(**user_input_encoded)

# 문장 임베딩 생성 (RoBERTa의 마지막 hidden state의 평균 사용)
sentence_embeddings = outputs.last_hidden_state.mean(dim=1)
user_input_embedding = user_output.last_hidden_state.mean(dim=1)

# 사용자 입력 문장과 각 판례 문장 간의 유사도 계산
similarities = torch.cosine_similarity(sentence_embeddings, user_input_embedding, dim=1).numpy()

# 상위 3개 유사한 사건의 인덱스 찾기
top_indices = np.argsort(similarities)[-3:][::-1]

# 상위 3개 유사한 사건 출력
for idx in top_indices:
    case_no = df.iloc[idx]['caseNo']
    similarity = similarities[idx]
    fact_one = df.iloc[idx]['fact_one']
    print(f'CaseNo: {case_no}, Similarity: {similarity}')
    print(f'Fact One: {fact_one}\n')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


CaseNo: 2017다216776, Similarity: 0.9425437450408936
Fact One: 제6차 개정 신용장통일규칙 에서 정한 ‘매입’의 의미와 방법 및 중개무역업자의 거래은행이 이른바 백투백신용장을 개설하면서 주신용장의 매입에 관한 약정을 체결한 것이 신용장 매입 방법 중 하나인 ‘현실적인 대가 지급을 갈음할 수 있는 무조건적인 대가 지급채무를 부담하는 것’에 해당하는지 여부일람출급 취소불능 화환신용장의 매입은행이 수익자에게 신용장 매입대금을 지급하여야 하는 시기은행에 의한 신용장 매입이 적법하지 않은 경우 신용장 개설은행이 신용장 거래에서 수익자의 사기행위가 밝혀졌다는 이유로 신용장의 만기에 서류를 제시한 위 은행에 대하여 신용장대금의 지급을 거절할 수 있는지 여부

CaseNo: 2011도1651, Similarity: 0.9418824911117554
Fact One: 공소사실이나 범죄사실의 동일성 여부 판단 기준피고인에 대하여 유죄판결이 확정된 ‘아파트 사전분양’으로 인한 구 주택건설촉진법 위반죄 범죄사실과 ‘아파트를 건축·분양할 의사나 능력 없이 피해자들을 기망하여 분양대금을 편취하였다’는 내용의 특정경제범죄 가중처벌 등에 관한 법률 위반 공소사실 사이에 동일성이 있다거나 두 죄가 1죄 내지 상상적 경합관계에 있다고 할 수 없다고 본 원심판단을 정당하다고 한 사례법원이 공소장변경절차를 거치지 아니하고 공소사실과 다른 사실을 인정하기 위한 요건피고인들이 특정경제범죄 가중처벌 등에 관한 법률 위반으로 기소된 사안에서 원심이 공소사실과 다른 내용으로 ‘피고인들의 임무’를 인정하였더라도 기본적 사실의 동일성 범위를 벗어나 새로운 임무를 인정하였다거나 이로 인해 피고인들의 방어권 행사에 실질적 불이익이 초래되었다고 볼 수 없다고 한 사례배임죄에서 ‘본인에게 손해를 가한 때’의 의미 및 위임받은 사무가 소유권이전등기의무인 경우 배임죄의 성립 요건피고인들이 담보권 실행을 위한 경매절차가 진행 중인 호텔을 피해자 측에 매도하면서 이에 관한 최